# Keras - Custom Loss

In [0]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation,BatchNormalization, Dropout, Input
from keras import optimizers
import tensorflow as tf


Using TensorFlow backend.


In [0]:
input_features = 10
output_features = 1

samples = 100
mean = 0
std = 1
X = np.random.normal(mean,std,size=(samples,input_features))
y = X[:,1]+X[:,4]
y = np.expand_dims(y,1)
print(X.shape,y.shape)


(100, 10) (100, 1)


In [0]:

learning_rate = 0.01
L1n = 2
L2n = 2
L3n = 4

inputx = Input((input_features,))
inputy = Input((output_features,))

x = Dense(L1n,activation='relu')(inputx)
x = BatchNormalization()(x)
L2 = Dense(L2n,activation='relu')(x)
L2o = Dense(output_features,activation='linear')(L2) #First output of the network

x = BatchNormalization()(L2)
x = Dense(L3n,activation='relu')(x)
x = BatchNormalization()(x)
FLo = Dense(output_features,activation='linear')(x) #Final output of the network

loss_1 = tf.keras.backend.abs(L2o-inputy)
loss_2 = tf.keras.backend.abs(FLo-inputy)
loss = tf.keras.backend.sum([loss_1,loss_2])

model = Model(inputs=[inputx,inputy], outputs=FLo)
model.add_loss(loss)

sgd = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd)




W0810 15:04:21.561161 140188943050624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 15:04:21.584468 140188943050624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 15:04:21.590009 140188943050624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 15:04:21.678297 140188943050624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0810 15:04:22.065088 

In [0]:
#Train test_
test_split=0.25
iterations=1000
eval_interval=100
eval_interval_train=200
batch_size = 20

test_n = int(np.ceil(samples*test_split))
train_n = samples - test_n

sample_range = list(np.arange(samples))

test_index = np.random.choice(sample_range,test_n,replace=False)
train_index = list(filter(lambda x: x not in test_index,sample_range))
X_train, X_test = X[train_index,:],X[test_index,:]
y_train, y_test = y[train_index,:],y[test_index,:]


for iterx in range(iterations):
  batch_index = np.random.choice(np.arange(train_n),batch_size,replace=False)
  x_train_b, y_train_b = X_train[batch_index,:],y_train[batch_index,:]
  train_metric = model.train_on_batch([x_train_b,y_train_b],None)
  if iterx%eval_interval ==0:
    eval_metric = model.evaluate([X_test,y_test])
    print("Testing:",iterx,eval_metric)
  if iterx%eval_interval_train ==0:
    eval_metric = model.evaluate([X_train,y_train])
    print("Training:",iterx,eval_metric)

W0810 15:05:37.399748 140188943050624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



25/25 [==============================] - 0s 4ms/step
Testing: 0 66.61975860595703
75/75 [==============================] - 0s 103us/step
Training: 0 67.06915913899739
25/25 [==============================] - 0s 27us/step
Testing: 100 61.083187103271484
25/25 [==============================] - 0s 25us/step
Testing: 200 61.29905700683594
75/75 [==============================] - 0s 109us/step
Training: 200 59.373212890625
25/25 [==============================] - 0s 29us/step
Testing: 300 61.12160110473633
25/25 [==============================] - 0s 25us/step
Testing: 400 62.86325454711914
75/75 [==============================] - 0s 75us/step
Training: 400 59.22956611633301
25/25 [==============================] - 0s 28us/step
Testing: 500 63.44997787475586
25/25 [==============================] - 0s 26us/step
Testing: 600 62.03165054321289
75/75 [==============================] - 0s 67us/step
Training: 600 59.10551518758138
25/25 [==============================] - 0s 29us/step
Testing: 70

# Save model and weights reparately


In [0]:
model.save('smodel.h5')

In [0]:
mx = keras.models.load_model('smodel.h5')

ValueError: ignored

In [0]:
json_string = model.to_json()
print(json_string)

{"class_name": "Model", "config": {"name": "model_1", "layers": [{"name": "input_1", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 10], "dtype": "float32", "sparse": false, "name": "input_1"}, "inbound_nodes": []}, {"name": "dense_1", "class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "units": 2, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["input_1", 0, 0, {}]]]}, {"name": "batch_normalization_1", "class_name": "BatchNormalization", "config": {"name": "batch_normalization_1", "trainable": true, "axis": -1, "momentum": 0.99, "epsilon": 0.001, "center": true, "scale": true, "beta_initializer": {"cl

In [0]:
json_model = 'jmodel.json'
with open(json_model,'w') as f:
  f.write(json_string)

In [0]:
model_weights_file = 'model_weights.h5'
model.save_weights(model_weights_file)

In [0]:
with open(json_model,'r') as f:
  json_loaded_string = f.read()

print(json_loaded_string)

{"class_name": "Model", "config": {"name": "model_1", "layers": [{"name": "input_1", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 10], "dtype": "float32", "sparse": false, "name": "input_1"}, "inbound_nodes": []}, {"name": "dense_1", "class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "units": 2, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["input_1", 0, 0, {}]]]}, {"name": "batch_normalization_1", "class_name": "BatchNormalization", "config": {"name": "batch_normalization_1", "trainable": true, "axis": -1, "momentum": 0.99, "epsilon": 0.001, "center": true, "scale": true, "beta_initializer": {"cl

In [0]:
modelx = keras.models.model_from_json(json_loaded_string)
modelx.load_weights(model_weights_file)